In [2]:
import sys
sys.path.append('../')

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config
import polars as pl

from model.transformers import (
	ConstantIntradayVolatilityEstimator,
	AverageDailyVolumeEstimator
)

set_config(transform_output='polars', enable_metadata_routing=True)

In [2]:
from pathlib import Path
p = Path(r'C:\Users\lachl\Documents\data\clean.arrow')
lf = pl.scan_ipc(p)

In [3]:
df = lf.collect()

In [6]:
window = 30

def set_col_routing(trans):
	return (
		trans
		.set_fit_request(cols=True)
		.set_transform_request(cols=True)
	)

vola_trans = set_col_routing(ConstantIntradayVolatilityEstimator(window, 'vola'))
volu_trans = set_col_routing(AverageDailyVolumeEstimator(window, 'volu'))

union = FeatureUnion([
	('original', 'passthrough'),
	('vola_trans', vola_trans),
	('volu_trans', volu_trans),
])

def drop_nulls(df: pl.DataFrame) -> pl.DataFrame:
	return df.drop_nulls()
drop_null_trans = FunctionTransformer(drop_nulls)

pipeline = Pipeline([
	('rolling_estimates', union),
	('drop_nulls', drop_null_trans),
])

cols ={
	'timestamp': 'timestamp',
	'price': 'price',
	'symbol': 'symbol'
}

In [10]:
data_df = pipeline.fit_transform(df, cols=cols)

In [4]:
from typing import Self, Iterable, Sequence
from sklearn.base import TransformerMixin, BaseEstimator, _fit_context # type: ignore
from sklearn.linear_model import LinearRegression, Ridge
import numpy as np

class OWModel(TransformerMixin, BaseEstimator):

  def __init__(
      self,
      half_life: int,
      locality: str,
      bin_duration: int,
      is_persistent: bool = False,
      regularization: float = 0,
  ) -> None:

    self.half_life = half_life
    self.locality = locality
    self.bin_duration = bin_duration
    self.is_persistent = is_persistent
    self.regularization = regularization

  _parameter_constraints = {
    'half_life': [int],
    'locality': [str],
    'bin_duration': [int],
    'is_persistent': [bool],
    'regularization': [float],
  }

  @property
  def alpha(self) -> float:
    """Represents the decay of the impact state from one bin to the next."""
    return 1 - np.exp(-np.log(2) / self.half_life * self.bin_duration)

  @property
  def over_cols(self) -> Iterable[str]:
    """
    Controls whether impact persists. We create a separate impact state 
    time series after grouping by the columns provided by this function.
    
    XXX: Does this present challenges when using TimeSeriesSplit in the
    persistent case since we *arbitrarily* set the impact to zero at the
    start of the block?!
    """
    return ['symbol', 'date'] if self.is_persistent else ['symbol']

  @property
  def ridge_alpha(self) -> float:
    # Only allow regularization in the case that we need it.
    return 0 if self.locality != 'mixed' else self.regularization

  def _col_change_expr(self, col: str) -> pl.Expr:
    # TODO: WHY THE HELL DO WE DIFF BY THIS AMOUNT OF STEPS?!
    return pl.col(col).diff(self.half_life // self.bin_duration).over(self.over_cols)
  
  @property
  def _model_coef_expr() -> pl.Expr:
    """Extracts the coefficient from the `model` column of `push_`."""
    return pl.col('model').map_elements(lambda m: m.coef_)

  @staticmethod
  def _fit_least_squares(
      cols: Sequence[pl.Series],
      ridge_alpha: float = 0,
  ) -> Ridge:
    """
    Fits a *potentially* regularized least squares model to the X
    and y values that have been packed into a struct in the first
    entry of cols.
    """

    structs = cols[0]
    X = structs.struct.field('X').to_numpy().reshape(-1, 1)
    y = structs.struct.field('y').to_numpy().reshape(-1, 1)
    return Ridge(alpha=ridge_alpha).fit(X, y) # type: ignore

  def _fit_local(self, impact_lf: pl.LazyFrame) -> pl.LazyFrame:
    """Fit a different regression for each symbol."""
    return (
      impact_lf.group_by('symbol', maintain_order=True)
      .agg(model=pl.map_groups('X_y', OWModel._fit_least_squares))
    )

  def _fit_global(self, impact_lf: pl.LazyFrame) -> pl.LazyFrame:
    """Fit a single regression by all symbols."""
    impact_lf = impact_lf.with_columns(symbol=pl.lit('all').cast(pl.Categorical))
    global_push = self._fit_local(impact_lf)
    n_symbols = self.symbols_.select(pl.len()).item(0, 0)
    return global_push.select(
      model=pl.repeat(pl.col('model'), n_symbols),
      symbol=self.symbols_.select('symbol').to_series()
    )

  def _fit_mixed(self, impact_lf: pl.LazyFrame) -> pl.LazyFrame:
    """
    Fit globally, then locally, shrinking the coefficients towards
    the global coefficient using L2 regularization.
    """
    # Global fitted parameter is beta_hat
    global_model = self._fit_global(impact_lf)
    impact_lf = (
      impact_lf.join(global_model, how='left', on='symbol')
      # TODO: clean this up with proper attribute accessor
      .with_columns(beta_hat=self._model_coef_expr)
      # L = sum (y_i - alpha - beta * x_i)^2 + lam * (beta - beta_hat)^2
      #   = sum (y_i - beta_hat * x_i - alpha - gamma * x_i)^2 + lam * gamma^2
      #   = sum (y_hat_i - alpha - gamma * x_i)^2 + lam * gamma^2
      # where gamma = beta - beta_hat, and y_hat_i = y_i - beta_hat * x_i
      .with_columns(d_price=pl.col('d_price') - pl.col('beta_hat') * pl.col('d_impact'))
    )
    gamma_lf = (
      self._fit_local(impact_lf)
      .join(global_model, how='left', on='symbol', r_suffix='global_')
      # TODO need to add beta_hat to all local beta estiamtes
      # .with_columns(pl.col('model').map_elements(lambda m: m.coef_ += 1))
    )
    return gamma_lf

  def _fit_push(self, impact_lf: pl.LazyFrame) -> pl.LazyFrame:
    """Delegates fitting to the correct method based on `self.locality`."""
    fns = {
      'local': self._fit_local,
      'global': self._fit_global,
      'mixed': self._fit_mixed,
    }
    try:
      return fns[self.locality](impact_lf).with_columns(push=self._model_coef_expr)
    except KeyError:
     raise ValueError(f'Unrecognised locality {self.locality}')

  def _compute_impact(
      self,
      lf: pl.LazyFrame,
      push: pl.LazyFrame,
  ) -> pl.LazyFrame:
    """Calculate impact state given some trades and push parameters."""
 
    return (
      lf.with_columns(pl.col('trade').mul('vola').truediv('adv'))

      # EWM = ax_n-1 + (1-a)x_n, but we want ax_n-1 + x_n to approximate
      # exponential decay of the impact state. So divide by 1-a first.
      .with_columns(norm=self.alpha)
      # Don't divide first impact state by 1-a.
      .with_columns(pl.col('norm').shift().over(self.over_cols).fill_null(1))
      .with_columns(pl.col('trade').truediv('norm'))

      .with_columns(pl.col('trade').ewm_mean(
       alpha=self.alpha, adjust=False, ignore_nulls=True
      ).over(self.over_cols))
      .join(push, how='left', on='symbol')
      .with_columns(impact=pl.col('trade').mul('push'))
    )
  
  def _set_up_least_squares(self, impact_lf: pl.LazyFrame, push: pl.LazyFrame) -> pl.LazyFrame:
    """
    Calculate impact with given push, as well as changes in impact and price
    to be used in a least squares calculation.
    """
    return (
      impact_lf.with_columns(
        # These columns are never used outside of the struct and so
        # will be optimized away! We leave them here as their aliases
        # improve readability.
        self._col_change_expr('impact').alias('d_impact'),
        self._col_change_expr('price').alias('d_price'),
      )
      .with_columns(X_y=pl.struct(X='d_impact', y='d_price'))
    )


  def _tidy_input(self, X: pl.DataFrame, cols: dict[str, str]) -> pl.LazyFrame:
    """Translate column names and add `date` column for convenience."""
    # TODO: Add schema validation here.
    tidy_lf = X.lazy().rename(cols)
    return tidy_lf.with_columns(date=pl.col('timestamp').dt.date())

  @_fit_context(prefer_skip_nested_validation=True) # type: ignore
  def fit(self, X: pl.DataFrame, y: None, cols: dict[str, str]) -> Self:
    tidy_lf = self._tidy_input(X, cols)

    # Calculate impact with unit push and calculate changes in impact
    # and absolute returns to perform a simple OLS of the form:
    # change in price = push * change in impact state + c.
    self.symbols_ = tidy_lf.select(pl.col('symbol').unique()).collect()
    push = self.symbols_.lazy().with_columns(push=1)
    impact_lf = self._compute_impact(lf, push)
    impact_with_change_lf = self._set_up_least_squares(impact_lf, push)

    self.push_ = self._fit_push(impact_with_change_lf).collect()
    # TODO: Should be moved to transform! This prevents saving so much data.
    # Maybe could cache at some point later for alpha backtesting efficiency!
    self.clean_price_ = (
      impact_lf.join(self.push_, how='left', on='symbol')
      .with_columns(price=pl.col('price') - pl.col('push') * pl.col('impact'))
      .select(['symbol', 'timestamp', 'price'])
    ).collect()

    return self

  def transform(self, X: pl.DataFrame, cols: dict[str, str]) -> pl.DataFrame:
    tidy_lf = self._tidy_input(X, cols)
    impact_lf = self._compute_impact(tidy_lf, self.push_)

    return (
      # Can maybe just app
      self.clean_price_.join(impact_lf, how='left', on=['symbol', 'timestamp'])
      .with_columns(price=pl.col('price') + pl.col('impact'))
    ).collect()

  def fit_transform( # type: ignore
      self,
      X: pl.DataFrame,
      y: None,
      cols: dict[str, str]
  ) -> pl.DataFrame:

    return self.fit(X, y, cols).transform(X, cols)

  def score(self, X: pl.LazyFrame, y: None) -> float:
    # TODO: Check if any new symbols were included
    tidy_lf = self._tidy_input(X, cols)
    tidy_lf.group_by('symbol', maintain_order=True).map_groups()

In [170]:
class MyTrans(TransformerMixin, BaseEstimator):

	def __init__(self, noise):
		self.noise = noise

	def fit(self, X, y):
		print('fit')
		self.reg_ = LinearRegression().fit(X, y + np.random.normal(0, self.noise, y.shape))
		self.score_ = self.reg_.score(X, y)
		return self
	
	def predict(self, X):
		return X

	def score(self, X, y):
		print('score')
		return self.score_

In [13]:
xx = pl.LazyFrame({'a': [1, 2]})
z = xx.select(pl.col('a').first())
yy = pl.LazyFrame({'b': [3, 4]})
yy.with_columns(a=pl.lit(z, allow_object=True)).collect()

b,a
i64,object
3,"naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan) SELECT [col(""a"").first()] FROM DF [""a""]; PROJECT */1 COLUMNS; SELECTION: None"
4,"naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan) SELECT [col(""a"").first()] FROM DF [""a""]; PROJECT */1 COLUMNS; SELECTION: None"


In [81]:
n = 1_000
x = np.random.normal(0, 1, n)
eps = np.random.normal(0, 1, n)
alpha, beta = 3, 10
y = alpha + beta * x + eps

In [84]:
test_df = (
	pl.LazyFrame({'xx': x, 'y': y})
	.with_row_index('idx')
	.with_columns(group=pl.when(pl.col('idx') < n // 2).then(0).otherwise(1))
)

def func(cols):
	# print(cols)
	X = cols[0].struct.field('x').to_numpy().reshape(-1, 1)
	y = cols[0].struct.field('y').to_numpy().reshape(-1, 1)
	return LinearRegression().fit(X, y)

test_df = (
	test_df
	.with_columns(struct=pl.struct(x='xx', y='y'))
	.group_by('group', maintain_order=True).agg(reg=pl.map_groups('struct', func))
).collect()

In [42]:
test_df

group,reg
i32,object
0,LinearRegression()
1,LinearRegression()


In [172]:
half_trans = FunctionTransformer(lambda x: x * 0.5)
fit_trans = MyTrans(noise=1)

In [173]:
pipe = Pipeline([
	('half_trans', half_trans),
	('fit_trans', fit_trans),
])

In [174]:
pipe.fit(x, y)

fit


Pipeline(steps=[('half_trans',
                 FunctionTransformer(func=<function <lambda> at 0x000002B45C80FC40>)),
                ('fit_trans', MyTrans(noise=1))])

In [175]:
from sklearn.model_selection import GridSearchCV

In [176]:
search = GridSearchCV(pipe, {'fit_trans__noise': [1, 10, 100]}, scoring='r2')

In [179]:
pipe.fit(x, y).get_params()['fit_trans'].score_

fit


0.9905965243195198

In [9]:
xx = pl.DataFrame({
  'a': [1, 2],
  'b': [LinearRegression().fit([[1], [2]], [1, 2]), LinearRegression().fit([[1], [2]], [1, 2])]
})
xx.select(pl.col('b').map_elements(lambda x: x.coef_))

C:\Users\lachl\AppData\Local\Temp\ipykernel_14084\3722549172.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  xx.select(pl.col('b').map_elements(lambda x: x.coef_))


b
f64
1.0
1.0


In [7]:
dir(pl.col('a').)

['__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_accessors',
 '_from_pyexpr',
 '_map_batches_wrapper',
 '_pyexpr',
 '_repr_html_',
 'abs',
 'add',
 'agg_groups',
 'alias',
 'all',
 'and_',
 'any',
 'append',
 'approx_n_unique',
 'arccos',
 'arccosh',
 'arcsin',
 'arcsinh'

In [9]:
reg = LinearRegression().fit([[1], [2]], [3, 4])
reg.coef_

array([1.])